# Querying Multiple Data Formats 
*By Winston Robson*

In this notebook, we will cover: 
- How to create DaskSQL tables from: 
    - Text file (CSV)
    - Apache Parquet 
    - cuDF DataFrame (GDF)
- How to `JOIN` multiple DaskSQL tables into one cuDF DataFrame with a single query.

#### Download Data
This cell will check if you have the data for this demo, and, if you don't, will download it for you.

In [1]:
import os

# relative path to data folder
data_dir = '../../../data/blazingsql/'

# does folder exist?
if not os.path.exists(data_dir):
    print('creating blazingsql directory\n')
    # create folder
    os.system('mkdir ../../data/blazingsql')

# do we have file 0?
if not os.path.isfile(data_dir + 'cancer_data_00.csv'):
    !wget -P ../../../data/blazingsql https://raw.githubusercontent.com/BlazingDB/bsql-demos/master/data/cancer_data_00.csv

# do we have file 1?
if not os.path.isfile(data_dir + 'cancer_data_01.parquet'):
    !wget -P ../../../data/blazingsql https://blazingsql-colab.s3.amazonaws.com/cancer_data/cancer_data_01.parquet

# do we have file 2?
if not os.path.isfile(data_dir + 'cancer_data_02.csv'):
    !wget -P ../../../data/blazingsql https://raw.githubusercontent.com/BlazingDB/bsql-demos/master/data/cancer_data_02.csv

## Create DaskContext
You can think of the DaskContext much like a Spark Context (i.e. information such as FileSystems registered & Tables created will be stored).

In [2]:
from dask_sql import Context
# start up DaskSQL
dc = Context()

#### Data Path
DaskSQL requires the full path to the data. This cell uses the `pwd` bash command to identify the path to this directory, then add it to the relative path to the notebooks-contrib `data` directory (i.e. what you'd type in Terminal to navigate to the data).

In [3]:
# bash command, returns SList w/ path (str) at 0th index
path = !pwd

# extract path to notebooks-contrib
path = path[0].split('getting_started_materials')[0] 

# add path to blazingsql data
path = path + 'data/blazingsql/'

# what's it look like?
path

'/home/u00ubc1kg5n2YAppSj357/notebooks-contrib/data/blazingsql/'

### Create Table from CSV
Here we create a DaskSQL table directly from a comma-separated values (CSV) file. 

In [4]:
# define column names and types
col_names = ['diagnosis_result', 'radius', 'texture', 'perimeter']
col_types = ['float32', 'float32', 'float32', 'float32'] 

# create table from CSV file
dc.create_table('data_00', path +'cancer_data_00.csv', gpu=True, names=col_names, dtype=col_types)


# df_result = dc.sql("SELECT * FROM data_00")
# df_result.head()
# type(df_result)
# df_result

### Create Table from Parquet
Here we create a DaskSQL table directly from an Apache Parquet file.

In [5]:
# create table from Parquet file
dc.create_table('data_01', path+'cancer_data_01.parquet', gpu= True)

### Create Table from GPU DataFrame
Here we use cuDF to create a GPU DataFrame (GDF), then use DaskSQL to create a table from that GDF.

The GDF is the standard memory representation for the RAPIDS AI ecosystem.

In [6]:
import cudf

# define column names & types
col_names = ['compactness', 'symmetry', 'fractal_dimension']
col_types = ['float32', 'float32', 'float32', 'float32']

# make GPU DataFrame from CSV w/ cuDF
gdf_02 = cudf.read_csv(path+'cancer_data_02.csv', names=col_names, dtype=col_types)

# create BlazingSQL table from cuDF DataFrame
dc.create_table('data_02', gdf_02)

In [7]:
import time

t0 = time.time()

# Join Tables Together 

Now we can use DaskSQL to join all three data formats in a single federated query. DaskSQL queries return results as a cuDF DataFrame.

In [8]:
# grab everything from data_00 & data_01 and area & smoothness from data_01
query = '''
        SELECT 
            a.*, 
            b.area, b.smoothness, 
            c.* 
        FROM 
            data_00 AS a
        LEFT JOIN data_01 AS b
            ON (a.perimeter = b.perimeter)
        LEFT JOIN data_02 AS c
            ON (b.compactness = c.compactness)
            '''

# join the tables together
join = dc.sql(query)

# display results (type(join)==cudf.core.dataframe.DataFrame)
join.head()

,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,1.0,12.0,13.0,94.0,578.0,0.113,0.229,0.207,0.077
1,1.0,22.0,19.0,97.0,645.0,0.105,0.187,0.225,0.069
2,0.0,17.0,11.0,87.0,566.0,0.098,0.081,0.189,0.058
3,0.0,16.0,14.0,86.0,563.0,0.082,0.060,0.178,0.056
4,1.0,10.0,24.0,97.0,645.0,0.105,0.187,0.225,0.069


In [9]:
t1 = time.time()
print(f"run_stuff took {t1-t0}s")

run_stuff took 2.06427001953125s


In [10]:
len(join.index)

316

# You're Ready to Rock
And... thats it! You are now live with DaskSQL.

Check out our [docs](https://dask-sql.readthedocs.io/) to get fancy or to learn more about how DaskSQL works with the rest of [RAPIDS AI](https://rapids.ai/).